## LSTM
- layer 4개

</br>
An Empirical exploration of recurrent
- RNN 구조 탐험

lstm odysey


## LRCN
- 이미지 순차적인 내용 분석 (video)
- 여기서 many to many
    - timed distribute 2번 써야된다.
    
Long-term Recurrent Convolutional Networks for Visual Recognition and Description

- frame 별로 나누어 CNN feature map을 LSTM에 입력

1. video input 구성
    - input shape : 4 dim( Seq , h ,w ,c)
    - many-to-many : TimeDistributed , return_squences=True
    - output shape :5 dim( batch, Seq h ,w ,c)
    
2. ConvNet : InceptionV3 Transfer Learning
    - option1. tf.keras.applications
    - option2. tensorflow_hub A. feature extraction/headless model/ feature vector B. Fine-tuning


In [1]:
import tensorflow as tf

C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [13]:
inputs = tf.keras.Input((None,224,224,3))
cnn = tf.keras.applications.inception_v3.InceptionV3(include_top=False
                                                    , pooling='avg')

In [14]:
tf.keras.utils.plot_model(cnn)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


### Lambda layer 
- functional paradigm을 이용해 layer에서 변경

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Lambda

- multi input, multi output
- layer에서 각각 적용, 모델에서는 한 번에 하기 위해 lambda사용

In [15]:
cnn_values = tf.keras.layers.TimeDistributed(tf.keras.layers.Lambda(lambda x:cnn(x)))(inputs)
lstm = tf.keras.layers.LSTM(256)(cnn_values)
outputs = tf.keras.layers.Dense(5)(lstm)

W1016 20:30:49.140893 13488 core.py:952] 
The following Variables were used a Lambda layer's call (lambda_4), but
are not present in its tracked objects:
  <tf.Variable 'conv2d_188/kernel:0' shape=(3, 3, 3, 32) dtype=float32>
  <tf.Variable 'batch_normalization_188/beta:0' shape=(32,) dtype=float32>
  <tf.Variable 'conv2d_189/kernel:0' shape=(3, 3, 32, 32) dtype=float32>
  <tf.Variable 'batch_normalization_189/beta:0' shape=(32,) dtype=float32>
  <tf.Variable 'conv2d_190/kernel:0' shape=(3, 3, 32, 64) dtype=float32>
  <tf.Variable 'batch_normalization_190/beta:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv2d_191/kernel:0' shape=(1, 1, 64, 80) dtype=float32>
  <tf.Variable 'batch_normalization_191/beta:0' shape=(80,) dtype=float32>
  <tf.Variable 'conv2d_192/kernel:0' shape=(3, 3, 80, 192) dtype=float32>
  <tf.Variable 'batch_normalization_192/beta:0' shape=(192,) dtype=float32>
  <tf.Variable 'conv2d_193/kernel:0' shape=(1, 1, 192, 64) dtype=float32>
  <tf.Variable 'batch_normaliza

In [16]:
model = tf.keras.models.Model(inputs, outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, None, 224, 224, 3 0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 2048)        0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               2360320   
_________________________________________________________________
dense (Dense)                (None, 5)                 1285      
Total params: 2,361,605
Trainable params: 2,361,605
Non-trainable params: 0
_________________________________________________________________


In [19]:
import numpy as np
y = np.random.random(5)
# y.reshape()

In [20]:
x = np.random.random((1,20,224,224,3))

In [23]:
model.fit(x,y.reshape(1,5),epochs=1)

1/1 [==============================] - 0s 7ms/step - loss: 3.6451


CONVOLUTIONAL, LONG SHORT-TERM MEMORY, FULLY CONNECTED DEEP NEURAL NETWORKS(2016)

---
## CRNN
- input 이미지 -> time distributed 불필요
- lstm 과 모양 맞추기 -> flatten, gap, reshape

ReNet: A Recurrent Neural Network Based Alternative to Convolutional Networks
     - CNN대신 RNN -> 성능 좋진 않고 가능성만

An End-to-End Trainable Neural Network for Image-based Sequence Recognition and Its Application to Scene Text Recognition

#### peephole
- 각 gate layer에 엿보기 구멍 추가해서 cell state 확인할수 있도록 함

### GRU
- forget gate, input gate를 하나로 합쳐서 update gate만듬
- cell state와 hidden 합침
- LSTM 간단하게

http://colah.github.io/posts/2015-08-Understanding-LSTMs/

---

## ConvLSTM
- LSTM자체에서 Convolution을 집어넣음.

example : https://keras.io/examples/vision/conv_lstm/

Convolutional LSTM Network: A Machine Learning Approach for Precipitation Nowcasting

In [24]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pylab as plt

In [25]:
seq = tf.keras.Sequential(
    [
        keras.Input(
            shape=(None, 40, 40, 1)
        ),  # Variable-length sequence of 40x40x1 frames
        layers.ConvLSTM2D(
            filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
        layers.BatchNormalization(),
        layers.Conv3D(
            filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
        ),
    ]
)

In [26]:
seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, None, 40, 40, 40)  59200     
_________________________________________________________________
batch_normalization_282 (Bat (None, None, 40, 40, 40)  160       
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 40, 40, 40)  115360    
_________________________________________________________________
batch_normalization_283 (Bat (None, None, 40, 40, 40)  160       
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, None, 40, 40, 40)  115360    
_________________________________________________________________
batch_normalization_284 (Bat (None, None, 40, 40, 40)  160       
_________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)  (None, None, 40, 40, 40)  1

action recognition.ipynb -> 동영상 행동 예측